# Lateral-torsional Rayleigh-Ritz equations

In [ ]:
import numpy as np
from sympy import *
from bikewheelcalc import BicycleWheel
from IPython.display import display

init_printing()

In [ ]:
# setup symbols
s = symbols('s', real=true)

# deformation
u = Function('u')
v = Function('v')
w = Function('w')
phi = Function('phi')

# Applied load
P = symbols('P', real=True)

# rim properties
R, EI2, GJ, EIw, mu, y0, r0 = symbols('R EI_2 GJ EI_w mu y_0, r_0',
                                      real=true, finite=true,
                                      positive=true)

t = symbols('t', real=true)  # Average net radial pull per unit length

# spoke properties
k_uu, k_up, k_pp = symbols('k_uu k_{u\phi} k_{\phi\phi}', real=true)

## Curvatures and total potential

In [ ]:
# curvatures
n = symbols('n', real=True, integer=True)
un, pn = symbols('u_n phi_n', real=True)
u = un*cos(n*s/R)
phi = pn*cos(n*s/R)

# Curvatures
k_2 = u.diff(s, 2) - phi/R
k_3 = u.diff(s)/R + phi.diff(s)

U_rim = (EI2*k_2**2 + GJ*k_3**2 + EIw*(k_3.diff(s))**2)/2

U_spokes = (k_uu*u**2 + 2*k_up*u*phi + k_pp*phi**2)/2

V_rim = R*t*(u.diff(s)**2 + (r0**2)*(phi.diff(s) - u.diff(s)/R)**2 +
         y0*(2*u.diff(s)*phi.diff(s) - phi**2/R))/2

PI = (integrate(U_rim + U_spokes - V_rim, (s, 0, 2*pi*R)).simplify()) - P*un

sub_r2 = [(y0**2, 0), (r0**2, 0)]
sub_y0 = [(y0, 0)]
PI = PI.subs(sub_r2 + sub_y0)

## Rayleigh-Ritz equations

In [ ]:
n_pos = symbols('n', real=True, integer=True, positive=True)
rr_eqns = [Eq(PI.subs(n, 0).diff(un)), Eq(PI.subs(n, 0).diff(pn)),
           Eq(PI.subs(n, n_pos).diff(un)), Eq(PI.subs(n, n_pos).diff(pn))]

display(rr_eqns[0])
display(rr_eqns[1])
display(rr_eqns[2])
display(rr_eqns[3])

## Solution

In [ ]:
s0 = solve(rr_eqns[0:2], (un, pn))

s0

In [ ]:
sn = solve(rr_eqns[2:4], (un, pn))

sn

## Mode stiffnesses

In [ ]:
1/s0[un]

In [ ]:
1/sn[un]